<a href="https://colab.research.google.com/github/dla9944/God_damn_deeplearning/blob/master/NEW_YORK_TIMES_comment_data_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pytorch 라이브러리
import torch
import torchvision
import torch.nn.functional as F
from torch.utils import data
from torch import nn, optim
from torchvision import transforms, datasets
from torchvision.utils import save_image

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
from matplotlib import cm 

import numpy as np
import pandas as pd

import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import random

In [2]:
from google.colab import files

In [7]:
from google.colab import files
!pip install -q kaggle #kaggle 설치 # --quiet

files.upload() 
!mkdir ~/.kaggle (~)
!cp kaggle.json ~/.kaggle/ y)
!chmod 600 ~/.kaggle/kaggle.json 

!kaggle datasets download -d aashita/nyt-comments
!ls 

Saving kaggle.json to kaggle (3).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
 98% 469M/480M [00:03<00:00, 177MB/s]
100% 480M/480M [00:03<00:00, 157MB/s]
'kaggle (1).json'  'kaggle (3).json'   nyt-comments.zip
'kaggle (2).json'   kaggle.json        sample_data


In [8]:
!unzip nyt-comments

Archive:  nyt-comments.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [9]:
import glob
from torch.utils.data.dataset import Dataset
import string

In [10]:
def clean_text(txt):
    new_text = []
    for v in txt:
        if v not in string.punctuation:
           new_text.append(v)
    return "".join(new_text).lower()

In [11]:
glob.glob('*.csv')

['ArticlesApril2017.csv',
 'CommentsMarch2018.csv',
 'ArticlesMarch2017.csv',
 'CommentsApril2018.csv',
 'CommentsMay2017.csv',
 'ArticlesJan2017.csv',
 'ArticlesMarch2018.csv',
 'CommentsApril2017.csv',
 'CommentsFeb2017.csv',
 'ArticlesFeb2017.csv',
 'CommentsJan2018.csv',
 'CommentsMarch2017.csv',
 'CommentsFeb2018.csv',
 'CommentsJan2017.csv',
 'ArticlesApril2018.csv',
 'ArticlesMay2017.csv',
 'ArticlesJan2018.csv',
 'ArticlesFeb2018.csv']

In [35]:
class TextGeneration(Dataset):
   def clean_text(self, txt):
       return "".join([v for v in txt if v not in string.punctuation]).lower()

   def __init__(self):
       super().__init__()
       all_headlines = []
       for filename in glob.glob('Articles*.csv'):
           article_df = pd.read_csv(filename)
           all_headlines.extend(list(article_df.headline.values))

       all_headlines = [h for h in all_headlines if h!='Unknown']
       self.corpus = [self.clean_text(x) for x in all_headlines]
       self.BOW = {}
       for line in self.corpus:
           for word in line.split():
               if word not in self.BOW.keys():
                  self.BOW[word] = len(self.BOW.keys())
       self.data = self.generate_sequence(self.corpus)
   def generate_sequence(self, txt):
       seq = []
       for line in txt:
           line = line.split()
           line_bow = [self.BOW[word] for word in line]
           data = [([line_bow[i], line_bow[i+1]], line_bow[i+2])
                   for i in range(len(line_bow)-2)]
           seq = seq+data
       return seq

   def __len__(self):
       return len(self.data)
   def __getitem__(self, i):
       data = np.array(self.data[i][0])
       label = np.array(self.data[i][1])
       return data, label

In [36]:
import torch.nn as nn
import torch

In [56]:
class LSTM(nn.Module):
   def __init__(self, num_embeddings):
       super().__init__()
       self.embed = nn.Embedding(
           num_embeddings = num_embeddings, embedding_dim = 16,
       )
       self.lstm = nn.LSTM(
           input_size = 16,
           hidden_size = 64,
           num_layers = 5,
           batch_first = True,
       )
       self.fc1 = nn.Linear(128, num_embeddings)
       self.fc2 = nn.Linear(num_embeddings, num_embeddings)
       self.relu = nn.ReLU()
       self.tanh = nn.Tanh()

   def forward(self, x):
       x = self.embed(x)
       x, _ = self.lstm(x)
       x = torch.reshape(x, (x.shape[0], -1))
       x = self.fc1(x)
       x = self.relu(x)
       x = self.fc2(x)
       return x

In [47]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.6 MB/s eta 0:00:00


In [48]:
# time check too
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from torch.optim.adam import Adam
from transformers.optimization import Adafactor, AdafactorSchedule

In [39]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')
print(f'running in {device}')

running in cuda


In [57]:
dataset = TextGeneration()
loader = DataLoader(dataset, batch_size = 64)

In [58]:
model = LSTM(num_embeddings = len(dataset.BOW)).to(device)
optim = torch.optim.AdamW(model.parameters(), lr = 0.0009, weight_decay = 0.01)

In [59]:
# START
for epoch in range(10):
    iterator = tqdm(loader)
    for data, label in iterator:
        optim.zero_grad()
        pred = model(torch.tensor(data, dtype = torch.long).to(device))
        label = torch.tensor(label, dtype = torch.long).to(device)
        loss = nn.CrossEntropyLoss()(pred, label)
        loss.backward()
        optim.step()
        iterator.set_description(f'epoch : {epoch} / loss : {loss.item()}')
torch.save(model.state_dict(), 'lstm.pt')                  

  0%|          | 0/677 [00:00<?, ?it/s]

<ipython-input-59-54c44b3e0d90>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = model(torch.tensor(data, dtype = torch.long).to(device))
<ipython-input-59-54c44b3e0d90>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype = torch.long).to(device)


  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

...

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]

  0%|          | 0/677 [00:00<?, ?it/s]